In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Thanks Shivam Bansal for keeping the data updated!

In [ ]:


from pandas_profiling import ProfileReport
#from dataprep.eda import create_report

from plotly.subplots import make_subplots
import plotly.graph_objects as go


from plotly.offline import plot, iplot, init_notebook_mode
import plotly.express as px
init_notebook_mode(connected=True)

#plotly did not import properly and was giving error. Using the solution from the reference doc : https://www.kaggle.com/product-feedback/138599

In [ ]:
#conda install dask==2021.12.0
#pip install -U dataprep
#from dataprep.eda import create_report

## Table of Content

* [Data Information](#section-one)
* [Handling Missing Data](#section-two)
* [Creating New Columns](#section-three)
* [Visualization](#section-four)
* [Conclusion](#section-five)

### Shortcuts that I used for the most common things in the notebook for easy flow:

a  - adding a cell above

b - adding the cell below

dd - delete the cell

'###', '##', '#' - used in markdown cells to make the heading slightly bigger

shift tab - to see the function descriptions

pip install packagename - Install new packages, make sure internet option on the right side is turn on

<a id="section-one"></a>
### Data Information

In [ ]:
# Reading the input file

In [ ]:
df = pd.read_csv('netflix1.csv')
df.head(10)

### Approach 1

In [ ]:
# To see the high level data details
df.info()


### Observations:

    1. The above code shows that there are some null values in the data
    
    2. Shows the total rows, name and number of columns and their datatypes
   

### Approach 2

In [ ]:
def missing_pct(df):
    # Calculate missing value and their percentage for each column
    missing_count_percent = df.isnull().sum() * 100 / df.shape[0]
    df_missing_count_percent = pd.DataFrame(missing_count_percent).round(2)
    df_missing_count_percent = df_missing_count_percent.reset_index().rename(
                    columns={
                            'index':'Column',
                            0:'Missing_Percentage (%)'
                    }
                )
    df_missing_value = df.isnull().sum()
    df_missing_value = df_missing_value.reset_index().rename(
                    columns={
                            'index':'Column',
                            0:'Missing_value_count'
                    }
                )
    # Sort the data frame
    #df_missing = df_missing.sort_values('Missing_Percentage (%)', ascending=False)
    Final = df_missing_value.merge(df_missing_count_percent, how = 'inner', left_on = 'Column', right_on = 'Column')
    Final = Final.sort_values(by = 'Missing_Percentage (%)',ascending = False)
    return Final

missing_pct(df)

The function ```missing_pct``` takes a data frame as an input and returns a data frame, where each row corresponds to a column in the original dataframe and contains column's name, number of missing values in that column as well as percentage of the missing values.

This is a standard template that I use for every dataset that I want to analyze.

In [ ]:
# Dataset information

# Approach 3
ProfileReport(df)

### Other libraries to see dataset information and automated basic charts are :

Pandas-Profiling

Sweetviz

Autoviz - this notebook used Autoviz for Netflix data https://www.kaggle.com/rsesha/autoviz-on-netflix-dataset?scriptVersionId=83019093

D-Tale

Source link : https://towardsdatascience.com/4-libraries-that-can-perform-eda-in-one-line-of-python-code-b13938a06ae
        
        

In [ ]:
#create_report(df)

<a id="section-two"></a>
# Handling the missing data and deleting duplicates

It is important to handle missing data because any statistical results based on a dataset with non-random missing values could be biased. So you really want to see if these are random or non-random missing values.

Drop the columns which has high number of missing values.

We can impute(filling the missing values using the available information such as mean, median) but we should carefully see the pattern of the column before doing imputation.

For example - You want to fill the height of a person who male. Simpley adding 0 in the missing column would not make sense. So we can take the averega of male height and use that value inplace of missing values.

1. Rating - manually filling the data usin data from Netflix website

2. Country - replacing blank countries with the most common country

3. Cast - replacing null value with "Data not available"

4. Director - replacing null value with "Data not available"

In [ ]:
# Rating data is mentioned incorrectly for few titles in the input file. Hence correcting it by checking the Maturity rating online

df['rating'] = df['rating'].replace({'74 min': 'TV-MA', '84 min': 'TV-MA', '66 min': 'TV-MA'})
df['rating'] = df['rating'].replace({'TV-Y7-FV': 'TV-Y7'})

In [ ]:
df['rating'].unique()

In [ ]:
# Renaming vaules for Rating for better understanding
# Source : https://help.netflix.com/en/node/2064
df['rating'] = df['rating'].replace({
                'PG-13': 'Teens - Age above 12',
                'TV-MA': 'Adults',
                'PG': 'Kids - with parental guidence',
                'TV-14': 'Teens - Age above 14',
                'TV-PG': 'Kids - with parental guidence',
                'TV-Y': 'Kids',
                'TV-Y7': 'Kids - Age above 7',
                'R': 'Adults',
                'TV-G': 'Kids',
                 'G': 'Kids',
                'NC-17': 'Adults',
                'NR': 'NR',
                'UR' : 'UR'
                
})

In [ ]:
df['rating'].unique()

In [ ]:
df['country'] = df['country'].fillna(df['country'].mode()[0])

df['cast'].replace(np.nan, 'No Data',inplace  = True)
df['director'].replace(np.nan, 'No Data',inplace  = True)
df.dropna(inplace=True)

# Drop Duplicates
df.drop_duplicates(inplace= True)

In [ ]:
# splitting the genres in different rows to use it in the viz later

#df_genre = df[df['title'].isin(['Blood & Water', 'Dick Johnson Is Dead', 'Ganglands' ])]
df_genre = df[['show_id', 'title','type', 'listed_in' ]]
df_genre = (df_genre.drop('listed_in', axis=1)
             .join
             (
             df_genre.listed_in
             .str
             .split(', ',expand=True)
             .stack()
             .reset_index(drop=True, level=1)
             .rename('listed_in')           
             ))


<a id="section-three"></a>
# Creating new columns


In [ ]:
# Creating new columns

df['month'] = pd.DatetimeIndex(df['date_added']).month

In [ ]:
# Total Shows and movies

df_count = df['show_id'].count().sum()
print(df_count)
# Split of showes and TV
df_type = df.groupby('type')['show_id'].count().reset_index()
df_type = df_type.rename(columns = {"show_id":"count_showids"})

<a id="section-four"></a>
# Visualization

In [ ]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Indicator(
    value = df_count))

fig = fig.update_layout(
        template = {'data' : {'indicator': [{
        'title': {'text': "Total content on Netflix"},}]
        }})

fig = fig.update_layout(
    #autosize=False,
    #width=500,
    height=100,
    margin=dict(l=50,r=50,b=0,t=1),)

# fig2 = px.pie(df_type, values='count_showids', names='type', color_discrete_sequence=px.colors.sequential.RdBu,
#        title='What type of titles are uploaded more on Netflix' , width=500, height=450)

fig.show()
#fig2.show()


fig = make_subplots(rows=1, cols=2, specs=[[{'type':'bar'}, {'type':'pie'}]])
fig.add_trace(
    
    go.Bar(x= df_type['count_showids'], y= df_type['type'], orientation = 'h', marker=dict(color=["Maroon", "Grey"]), showlegend=False, 
           text = df_type['count_showids'], textposition='auto'),
    row=1, col=1)

fig.add_trace(
    
    go.Pie(labels=df_type['type'], values=df_type['count_showids'], marker_colors= ["Maroon", "Grey"]),
    row=1, col=2)

fig.update_layout(
    title_text="'What type of content is more uploaded more on Netflix?")
fig.show()


We observe that there are more movies than TV shows on Netflix

In [ ]:
# splitting the countries in different rows 
#df_genre = df[df['title'].isin(['Blood & Water', 'Dick Johnson Is Dead', 'Ganglands' ])]
df_country = df[['show_id', 'title','type', 'country' ]]
df_country = (df_country.drop('country', axis=1)
             .join
             (
             df_country.country
             .str
             .split(', ',expand=True)
             .stack()
             .reset_index(drop=True, level=1)
             .rename('country')           
             ))


In [ ]:
df_country_viz_total = df_country[["title", "country"]]
df_country_viz_total = df_country_viz_total.groupby(['country'])["title"].count().reset_index().sort_values('title', ascending= False).head(10)
df_country_viz_total = df_country_viz_total.rename(columns = {"title": "movies_count",})

 
fig1 = px.bar(df_country_viz_total, x='country', y='movies_count', color_discrete_sequence=px.colors.sequential.RdBu,
       title='Top 10 countries with Netflix Content ')


df_country_viz = df_country[["title", "country"]]
df_country_viz = df_country_viz.groupby(['country'])["title"].count().reset_index().sort_values('title', ascending= False).head(10)

df_country_viz1 = df_country[["title", "type", "country"]]
df_country_viz1 = df_country_viz1.groupby(['country', 'type'])["title"].count().reset_index().sort_values('title', ascending= False)
df_country_viz1 = df_country_viz1.rename(columns = {"title": "movies_count",})

final1 = df_country_viz.merge(df_country_viz1, how = 'left', left_on = 'country', right_on = 'country')
final1['percentage'] = (final1['movies_count']/final1['title'])*100
final1['percentage'] = final1['percentage'].round(1)
final1['percent_string'] = final1['percentage'].astype(str)+ '%'


fig2 = px.bar(final1, x='country', y='percentage', color = 'type',
       title='Top 10 countries with Movie/TV show split ')

In [ ]:
fig = go.Figure()
fig.add_trace(
    
go.Bar(x= df_country_viz_total['country'], y= df_country_viz_total['movies_count'], marker_color = 'Maroon',
           text = df_country_viz_total['movies_count'], textposition='auto'))

fig.update_layout(title_text = "Top 10 countries with Netflix Content"
                  , yaxis=dict(title='Movies/TV Shows Count'))
fig.show()

final_movie = final1.query("type == 'Movie'")
final_show = final1.query("type == 'TV Show'")

fig = go.Figure()
fig.add_trace(go.Bar(
    x=  final_movie['country'],
    y= final_movie['percentage'],
    showlegend=True,
    text = final_movie['percent_string'], 
    textposition='auto',
    name='Movie',
    marker_color='Maroon'    
    
))
fig.add_trace(go.Bar(
    x= final_show['country'],
    y= final_show['percentage'],
    showlegend=True,
    text = final_show['percent_string'], 
    textposition='auto',
    name='TV Show',
    marker_color='Grey' 
))



# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='stack', title_text = 'Top 10 countries with Movie/TV show split '
                  , yaxis=dict(title='% Movies/TV Shows Count'))
fig.show()




As expected, 
1. Unites States tops the chart followed by India, United Kingdom, and Canada.
2. Interestingly, the content available in India is heavily skewed towards movies, confirming the intuition about big influence of bollywood in-house movie production. 
3. South Korea has the highest percentage of TV shows

In [ ]:
# df_country_viz = df_country[["title", "type", "country"]]
# df_country_viz = df_country_viz.groupby(['country', 'type'])["title"].count().reset_index().sort_values('title', ascending= False)
# df_country_viz = df_country_viz.rename(columns = {"title": "movies_count",})

# df_country_movie = df_country_viz.query("type == 'Movie'").head(10) 
# fig1 = px.bar(df_country_movie, x='country', y='movies_count',color_discrete_sequence=['Maroon'],
#        title='Top 10 countries with the most Netflix movies')

# df_country_movie = df_country_viz.query("type == 'TV Show'").head(10)
# fig2 = px.bar(df_country_movie, x='country', y='movies_count', color_discrete_sequence=['gray'],
#        title='Top 10 countries with the most Netflix TV Shows')

# fig1.show()
# fig2.show()

United States is the top leaader in both movie and TV shows. India followed US in the overall content and it seems that it has the most number of movies with very less percentage of TV shows comapred to UK and Japan.

In [ ]:
df_2 = df.query("type == 'Movie'")
df_2 = df_2[["title", "rating"]]
df_2 = df_2.groupby(['rating'])["title"].count().reset_index().sort_values('title', ascending = False)
df_2 = df_2.rename(columns = {"title": "movies_count"})
px.bar(df_2, x='rating', y='movies_count', color_discrete_sequence=px.colors.sequential.RdBu,
       title='For which category the maximum content(Movies) are uploaded? ')


It seems the most content(Movies) on Netflix caters to Adults and then teens.

In [ ]:
df_3 = df.query("type == 'TV Show'")
df_3 = df_3[["title", "rating"]]
df_3 = df_3.groupby('rating')["title"].count().reset_index().sort_values('title', ascending = False)
df_3 = df_3.rename(columns = {"title": "movies_count"})
px.bar(df_3, x='rating', y='movies_count', color_discrete_sequence=['grey'],
       title='For which category the maximum content(TV Shows) are uploaded?')


It seems the most content(TV shows) on Netflix caters to Adults and then teens.

In [ ]:
df_5 = df.query("release_year >= 2007")
df_5 = df_5.groupby("release_year")["show_id"].count().reset_index()

fig = px.area(df_5, x='release_year', y='show_id', color_discrete_sequence=px.colors.sequential.RdBu,
      title='Overall content release Trend')
fig.show()

In 2007, Netflix introduced streaming media and video on demand. We see a slow in the beginning but then it picked up in 2014-2015 and there is a rapid increase till 2018.

By 2018, the content on netlix was 13 times of 2007 year's content. But it has declined since 2019 since the beginning of covid. The other factor could be - In 2019, Disney plus was also launched. Films and television series produced by The Walt Disney Studios and Walt Disney Television, such as Marvel movies moved to Disney plus.

In [ ]:
# #df_4 = df.query("type == 'Movie'")
# df_4 = df.query("release_year >= 2007")
# df_4 = df_4.groupby(["type","release_year"])["show_id"].count().reset_index()

# fig  = px.line(df_4, x='release_year', y='show_id', color = 'type',
#       title='Movies/TV Show release yearly Trend')
# fig.show()

In [ ]:
#df_4 = df.query("type == 'Movie'")
df_4 = df.query("release_year >= 2007")
df_4 = df_4.groupby(["type","release_year"])["show_id"].count().reset_index()
df_4_movie = df_4.query("type == 'Movie'")
df_4_show = df_4.query("type == 'TV Show'")

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=  df_4_movie['release_year'],
    y= df_4_movie['show_id'],
    showlegend=True,
    text = df_4_movie['show_id'], 
    
    name='Movie',
    marker_color='Maroon'    
    
))
fig.add_trace(go.Scatter(
    x=  df_4_show['release_year'],
    y= df_4_show['show_id'],
    showlegend=True,
    text = df_4_show['show_id'], 
 
    name='TV Show',
    marker_color='Grey' 
))

fig.update_traces( mode='lines+markers')
fig.update_layout(title_text = 'Movies/TV Show release yearly Trend' )
fig.show()

It seems like Netflix focused on movies, and the movie count increases significantly till 2018. There's been a decline in the movies count but a steady growth in the TV shows since 2018. 

In [ ]:
# df_4 = df.query("release_year >= 2007")

# df_4 = df_4[["type","month",'release_year', "show_id"]]
# df_4 = df_4.groupby(['release_year', 'month', 'type'])['show_id'].count().reset_index()
# df_4 = df_4.rename(columns = {"show_id": "total_shows"})
# df_4 = df_4.groupby(['month', 'type'])['total_shows'].mean().reset_index()



# fig  = px.line(df_4, x='month', y='total_shows', color = 'type',
#       title='All years Movies/TV Show release Month Trend')
# fig.show()

In [ ]:
df_4 = df.query("release_year >= 2007")

df_4 = df_4[["type","month",'release_year', "show_id"]]
df_4 = df_4.groupby(['release_year', 'month', 'type'])['show_id'].count().reset_index()
df_4 = df_4.rename(columns = {"show_id": "total_shows"})
df_4 = df_4.groupby(['month', 'type'])['total_shows'].mean().reset_index()

df_4_movie = df_4.query("type == 'Movie'")
df_4_show = df_4.query("type == 'TV Show'")

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=  df_4_movie['month'],
    y= df_4_movie['total_shows'],
    showlegend=True,
    text = df_4_movie['total_shows'], 
    name='Movie',
    marker_color='Maroon'    
    
))
fig.add_trace(go.Scatter(
    x=  df_4_show['month'],
    y= df_4_show['total_shows'],
    showlegend=True,
    text = df_4_show['total_shows'], 
    name='TV Show',
    marker_color='Grey' 
))

fig.update_traces( mode='lines+markers')
fig.update_layout(title_text = 'Movies/TV Shows average release monthly trend' )
fig.show()



It appears that there is no specific pattern in which we could see more movies are added to particular months,

In [ ]:
def trend_yearwise(year):
    
    title = (f'Movies/TV Show release Month Trend for year {year}' )
    df_6 = df.query("release_year == @year")
    df_6 = df_6.groupby(["type","month"])["show_id"].count().reset_index()
    df_6_movie = df_6.query("type == 'Movie'")
    df_6_show = df_6.query("type == 'TV Show'")
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(
    x=  df_6_movie['month'],
    y= df_6_movie['show_id'],
    showlegend=True,
    text = df_6_movie['show_id'], 
    name='Movie',
    marker_color='Maroon'    

    ))
    fig.add_trace(go.Scatter(
    x=  df_6_show['month'],
    y= df_6_show['show_id'],
    showlegend=True,
    text = df_6_show['show_id'], 
    name='TV Show',
    marker_color='Grey' 
    ))

    fig.update_traces( mode='lines+markers')
    fig.update_layout(title_text =  title )
    fig.show()
    
trend_yearwise(2019)    

In [ ]:
df_genre_viz = df_genre[["title", "type", "listed_in"]]
df_genre_viz = df_genre_viz.groupby(['listed_in', 'type'])["title"].count().reset_index().sort_values('title')
df_genre_viz = df_genre_viz.rename(columns = {"title": "movies_count", "listed_in": "Genre"})

df_genre_movie = df_genre_viz.query("type == 'Movie'")
df_genre_tvshow = df_genre_viz.query("type == 'TV Show'")
# fig1 = px.bar(df_genre_movie, x='movies_count', y='Genre', color_discrete_sequence=px.colors.sequential.RdBu,
#        title='For which Genre the maximum content(Movies) are uploaded? ', height=600)
# fig2 = px.bar(df_genre_tvshow, x='Genre', y='movies_count', color_discrete_sequence=['Grey'],
#        title='For which Genre the maximum content(Shows) are uploaded? ')
#fig1.show() 
#fig2.show()
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'bar'}, {'type':'bar'}]],
                   subplot_titles = ['For which Genre the maximum Movies are uploaded?', 'For which Genre the maximum Shows are uploaded? ' ],
                   horizontal_spacing = 0.3)
fig.add_trace(
    
    go.Bar(x= df_genre_movie['movies_count'], y= df_genre_movie['Genre'], orientation = 'h', marker_color='Maroon',  name="Movie",
           text = df_type['count_showids'], textposition='auto'),
    row=1, col=1)

fig.add_trace(
    
    go.Bar(x= df_genre_tvshow['movies_count'], y= df_genre_tvshow['Genre'], orientation = 'h', marker_color = 'Grey', name = "TV Show"),
    row=1, col=2)

fig.update_layout( height = 600)
fig.show()

It looks like for both TV shows and movies - the three most common genres are international movies and dramas.

In [ ]:
df_9 = df.query("type == 'TV Show'")
df_9 = df_9[[ "title", "duration"]]
df_9 = df_9.groupby(['duration'])["title"].count().reset_index().sort_values('title', ascending = False)
#df_9 = df_9['duration'].replace("seasons", "")
df_9 = df_9.rename(columns = {"title": "TV Shows", "duration" : "Seasons"})


df_10 = df.query("type == 'Movie'")
df_10['duration'] = df_10['duration'].fillna("0")
df_10['duration'] = df_10['duration'].str.split(" ").str[0].astype(int)




fig_show = px.bar(df_9, x='Seasons', y='TV Shows', color_discrete_sequence=['grey'],
       title='TV Shows seasons ')
fig_Movie = px.histogram(df_10, x="duration" , nbins = 20, color_discrete_sequence=px.colors.sequential.RdBu
                  , title = "Movie Duration")

fig_Movie.show()
fig_show.show()


The duration for most movies on netflix falls between 80-120 mins with very few movies more than 150 mins.

Most shows on Netflix has only season1.

<a id="section-five"></a>
# Conclusion

We did exploratory data analysis on Netflix Movie Data. We found a lot of insights from the data. This is the first step in our series.  

Next, we will engineer useful features and begin developing our recommendation model.

For modelling step, we will start with the rating data from here : https://www.kaggle.com/netflix-inc/netflix-prize-data . Then we will see if the extended movie features from this dataset helps to improve our recommendation system.

Stay tuned! 


In [ ]:
#Work in progress